In [35]:
import torch
import torchaudio
import os
import numpy as np
import pandas as pd
import json

DIR = '../audioData/sythenticSoundscenes/test/'

In [14]:
all_files = os.listdir('../audioData/sythenticSoundscenes/test/')
aud_file_list = []

for file in all_files:
    if file.split('.')[-1] == 'wav':
        aud_file_list.append(file)



3000

In [20]:
lab_file_list = []
for file in all_files:
    if file.split('.')[-1] == 'jams':
        lab_file_list.append(file)

(3000,
 ['bus10_0.jams',
  'bus10_1.jams',
  'bus10_2.jams',
  'bus10_3.jams',
  'bus10_4.jams',
  'bus10_5.jams',
  'bus10_6.jams',
  'bus10_7.jams',
  'bus10_8.jams',
  'bus10_9.jams',
  'bus10_downshift_0.jams',
  'bus10_downshift_1.jams',
  'bus10_downshift_2.jams',
  'bus10_downshift_3.jams',
  'bus10_downshift_4.jams',
  'bus10_downshift_5.jams',
  'bus10_downshift_6.jams',
  'bus10_downshift_7.jams',
  'bus10_downshift_8.jams',
  'bus10_downshift_9.jams',
  'bus10_upshift_0.jams',
  'bus10_upshift_1.jams',
  'bus10_upshift_2.jams',
  'bus10_upshift_3.jams',
  'bus10_upshift_4.jams',
  'bus10_upshift_5.jams',
  'bus10_upshift_6.jams',
  'bus10_upshift_7.jams',
  'bus10_upshift_8.jams',
  'bus10_upshift_9.jams',
  'bus1_0.jams',
  'bus1_1.jams',
  'bus1_2.jams',
  'bus1_3.jams',
  'bus1_4.jams',
  'bus1_5.jams',
  'bus1_6.jams',
  'bus1_7.jams',
  'bus1_8.jams',
  'bus1_9.jams',
  'bus1_downshift_0.jams',
  'bus1_downshift_1.jams',
  'bus1_downshift_2.jams',
  'bus1_downshift_3.ja

In [ ]:
import torch.utils.data import Dataset

class scraperDataset(Dataset):

    def __init__(self, dataset_csv, transform=None):

        self.dataset_csv = dataset_csv
        self.transform = transform
        self.dataframe = pd.read_csv(dataset_csv)

        self.audio_files = self.dataframe['audio_fileNames']
        self.label_files = self.dataframe['label_fileNames']

    def __len__(self):
        return len(self.audio_files)

In [22]:
file_count = pd.DataFrame()
file_count['audio_fileNames'] = aud_file_list
file_count['label_fileNames'] = lab_file_list


,audio_fileNames,label_fileNames
0,bus10_0.wav,bus10_0.jams
1,bus10_1.wav,bus10_1.jams
2,bus10_2.wav,bus10_2.jams
3,bus10_3.wav,bus10_3.jams
4,bus10_4.wav,bus10_4.jams
...,...,...
2995,tubestation9_upshift_5.wav,tubestation9_upshift_5.jams
2996,tubestation9_upshift_6.wav,tubestation9_upshift_6.jams
2997,tubestation9_upshift_7.wav,tubestation9_upshift_7.jams
2998,tubestation9_upshift_8.wav,tubestation9_upshift_8.jams


In [33]:
sam_file = '../audioData/sythenticSoundscenes/test/restaurant8_downshift_0.wav'

def get_label_data_json(audio_filename):
    dir_name, fname = os.path.split(audio_filename)
    lab_fname = os.path.join(dir_name, (fname.split('.')[0] + '.jams'))
    with open(lab_fname, 'r') as f:
        label_data_json = json.load(f)

    return label_data_json

def get_justLabels(audio_filename, event_labels=True):
    scene_label = []
    event_label = []
    label_data_json = get_label_data_json(audio_filename)
    
    for events in label_data_json['annotations'][0]['data']:
        event_label.append(events['value']['label'])
    scene_label.append(event_label.pop(0))
    #print(scene_label, event_label)
    if event_labels:
        return scene_label, event_label
    else:
        return scene_label

def get_labels_with_timestamps(audio_filename):
    events_dict = {}
    label_data_json = get_label_data_json(audio_filename)

    for events in label_data_json['annotations'][0]['data']:
        start_time = events['value']['event_time']
        duration = events['value']['event_duration']
        events_dict[events['value']['label']] = [start_time, duration]

    return events_dict

get_justLabels(sam_file, event_labels=False)

['restaurant8']

In [42]:
activity_labels_list = []
for file in aud_file_list:
    activity_labels_list.append(get_justLabels(os.path.join(DIR,file), event_labels=False)[0])

In [43]:
import re

pattern = re.compile(r'\d+')

for i in range(0,len(activity_labels_list)):
    activity_labels_list[i] = pattern.sub('', activity_labels_list[i])

activity_labels_list[-1]

'tubestation'

In [44]:
file_count['acoustic_scene_label'] = activity_labels_list
file_count

,audio_fileNames,label_fileNames,acoustic_scene_label
0,bus10_0.wav,bus10_0.jams,bus
1,bus10_1.wav,bus10_1.jams,bus
2,bus10_2.wav,bus10_2.jams,bus
3,bus10_3.wav,bus10_3.jams,bus
4,bus10_4.wav,bus10_4.jams,bus
...,...,...,...
2995,tubestation9_upshift_5.wav,tubestation9_upshift_5.jams,tubestation
2996,tubestation9_upshift_6.wav,tubestation9_upshift_6.jams,tubestation
2997,tubestation9_upshift_7.wav,tubestation9_upshift_7.jams,tubestation
2998,tubestation9_upshift_8.wav,tubestation9_upshift_8.jams,tubestation


In [45]:
file_count['acoustic_scene_label'].unique()

array(['bus', 'busystreet', 'office', 'openairmarket', 'park',
       'quietstreet', 'restaurant', 'supermarket', 'tube', 'tubestation'],
      dtype=object)

In [7]:
sam_file = '../audioData/sythenticSoundscenes/test/bus1_0.jams'

with open(sam_file, 'r') as f:
    data = json.load(f)


#data = json.dumps(data)
for i in data['annotations'][0]['data']:
    print(i['value']['label'])

bus1
speech
laughter
keys
cough
speech
phone
speech
clearthroat
clearthroat
speech
laughter
speech
cough
speech
phone
cough


In [45]:
data

{'annotations': [{'annotation_metadata': {'curator': {'name': '', 'email': ''},
    'annotator': {},
    'version': '',
    'corpus': '',
    'annotation_tools': '',
    'annotation_rules': '',
    'validation': '',
    'data_source': ''},
   'namespace': 'scaper',
   'data': [{'time': 0.0,
     'duration': 30.0,
     'value': {'label': 'bus1',
      'source_file': 'inputSounds\\background\\train\\bus1\\bus01.wav',
      'source_time': 0,
      'event_time': 0,
      'event_duration': 30.0,
      'snr': 0,
      'role': 'background',
      'pitch_shift': None,
      'time_stretch': None},
     'confidence': 1.0},
    {'time': 8.40278405578049,
     'duration': 1.2493071687393194,
     'value': {'label': 'speech',
      'source_file': 'inputSounds\\foreground\\testing\\speech\\speech_norm_001.wav',
      'source_time': 0.0,
      'event_time': 8.40278405578049,
      'event_duration': 1.3222545819640197,
      'snr': 6.233151927278712,
      'role': 'foreground',
      'pitch_shift': 0.